In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import *

In [39]:
sub='401'
subids = ['401']
behavestring = '/Volumes/Voss_Lab/EEG/domcueMIXED_djb/behave.data/'

def set_behavior_path(sub, behavestring):
    extra='study'
    behaveobj=[behavestring+sub+extra]
    behavepath=Path(behaveobj[0])
    behavepath.exists()
    return behavepath

def read_behave_file(filepath):
    """read in behavearray, turn into DataFrame and delete extra columns"""
    colnames=['obj1','obj2','obj3','cuecond','loc1','x1','y1','loc2','x2','y2','loc3','x3','y3',
     'domRT','dom_resp','cond','block','tmp', 'tmp']
    behavearray=pd.read_table(filepath,header=None,names=colnames)
    tmpmask=~behavearray.columns.str.contains('tmp')
    behavearray=behavearray[behavearray.columns[tmpmask]]
    trialnum = np.arange(1,len(behavearray)+1)
    behavearray['studytrial'] = trialnum
    return behavearray


behavepath = set_behavior_path(sub, behavestring)
behavearray = read_behave_file(behavepath)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py:709: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [40]:
def adjust_pres_coords(array,x,y,xmax=1920/2,ymax=1080/2):
    """adjustment for behavioral coords to match
    eye coords for presentation version of exp"""
    newarray=pd.DataFrame()
    newarray[x]=array[x]+xmax
    newarray[y]=(array[y]-ymax)*-1
    return newarray

def apply_adjust_pres_coords(behavearray):
    """applies adjust_pres_coords to all
    coords in behave array"""
    newloc1=adjust_pres_coords(behavearray,'x1','y1')
    newloc2=adjust_pres_coords(behavearray,'x2','y2')
    newloc3=adjust_pres_coords(behavearray,'x3','y3')
    newlocs=pd.concat([newloc1,newloc2,newloc3],axis=1)

    cols=newlocs.columns.tolist()
    for loc in cols:
        behavearray[loc]=newlocs[loc]
    return behavearray

behavearray = apply_adjust_pres_coords(behavearray)


In [41]:
eyestring = '/Volumes/Voss_Lab/EEG/domcueMIXED_djb/eye.data/'
eyepath=Path(eyestring)

def parse_eye_filename(pathobject):
    fname=pathobject.name
    parts=fname.split(".")[0]
    subject=parts[:3]
    block=parts[3:4]
    subdict={"subject":subject, "block":block, "fname":fname}
    return subdict

def get_eye_files(subids,eyepath):
    """ returns master dataframe including eye file name, block, phase, subid
    input list of subject strings, Path object pointing to eye files
    """
    print(subids)
    substrings=[s+"*.asc" for s in subids]
    subinfo=[]
    for s in substrings:
        for filepathobj in eyepath.glob(s):
            subdict=parse_eye_filename(filepathobj)
            subinfo.append(subdict)

    masterdf=pd.DataFrame(subinfo).sort_values(by=["subject","block"])
    print(masterdf.head())
    masterdf=masterdf[["subject","block","fname"]]
    masterdf.index=range(len(masterdf))
    return masterdf

masterdf = get_eye_files(subids,eyepath)
masterdf

['401']
  block     fname subject
2     1  4011.asc     401
3     2  4012.asc     401
5     3  4013.asc     401
6     4  4014.asc     401
4     5  4015.asc     401


,subject,block,fname
0,401,1,4011.asc
1,401,2,4012.asc
2,401,3,4013.asc
3,401,4,4014.asc
4,401,5,4015.asc
5,401,6,4016.asc
6,401,7,4017.asc
7,401,8,4018.asc


In [85]:
def parse_eye_events_to_intline(line,extrainfo):
    efixspace=["","",""]
    eblinkspace=efixspace*2
    newline=line.split()
    if "EFIX" in line:
        newline.extend(efixspace)
    elif "EBLINK" in line:
        newline.extend(eblinkspace)
    newline.extend(extrainfo)
    return newline

def parse_eye_line(eye_sub, eyestring):
    """ parses each line of eye file for a given eye_phase_sub
    input one phase type list of files for a subs
    and the path to the file (in form of a string)
    outputs dataframe with all events in table
    """
    etypes=('ESACC','EFIX','EBLINK')
    study=[]
    restudy=[]
    blocks=eye_sub.block
    fnames=eye_sub.fname
    subjects=eye_sub.subject
    totalcount=0
    trialnum=0
    print(eyestring)
    for block,fname,subject in zip(blocks,fnames,subjects):
        path_file=eyestring+fname
        startcount=0
        p=Path(path_file)
        with p.open() as f:
            for line in f:
                if "START" in line:
                    totalcount=totalcount+1
                    startline=line.split()
                    starttime=int(startline[1])
                    if totalcount % 2:
                        trialnum = trialnum+1

                if any(e in line for e in etypes):
                    extrainfo=[starttime,trialnum,block,subject]
                    newline=parse_eye_events_to_intline(line,extrainfo)
                    if totalcount % 2:
                        restudy.append(newline)
                    else:
                        study.append(newline)
            print(trialnum, block, startcount)
    return study, restudy
study, restudy = parse_eye_line(eye_sub, eyestring)


def events_to_df(events):
    """ change raw events to data DataFrame
    then and change values to numeric"""

    eye_events_df=pd.DataFrame(events)
    eye_events_df=eye_events_df.apply(pd.to_numeric,errors='ignore')
    headers=["event","eye","start","end","duration",
    "xstart","ystart","xend","yend","?","?","trialstart",
    "trialnum","block","sub"]
    eye_events_df.columns=headers
    return eye_events_df

def eventsdf_cleanup(eye_events_df):
    x=pd.DataFrame()
    """adjust trial start time, remove irrelevant values in fixation rows,
    and then delete excess columns"""

    eyedf_clean=eye_events_df.copy()

    eyedf_clean['start']=eyedf_clean['start']-eyedf_clean['trialstart']
    eyedf_clean['end']=eyedf_clean['end']-eyedf_clean['trialstart']

    efix_mask = (eyedf_clean["event"]=="EFIX")
    eyedf_clean.loc[efix_mask, 'xend'] = np.nan
    for col in ['xstart', 'ystart', 'xend', 'yend']:
        x[col] = pd.to_numeric(eyedf_clean[col], errors='coerce')

    del eyedf_clean['trialstart']
    del eyedf_clean['?']
    del eyedf_clean['eye']

    return eyedf_clean, x

studydf = events_to_df(study)
studyeyearray, xs = eventsdf_cleanup(studydf)

restudydf = events_to_df(restudy)
restudeyearray, xr = eventsdf_cleanup(restudydf)

/Volumes/Voss_Lab/EEG/domcueMIXED_djb/eye.data/
20 1 0
40 2 0
60 3 0
80 4 0
100 5 0
120 6 0
140 7 0
160 8 0


In [88]:
studyeyearray
xs.loc[2158]

xstart       NaN
ystart       NaN
xend      1547.7
yend       686.2
Name: 2158, dtype: float64

In [59]:
studyeyearray['phase']='study'
restudeyearray['phase']='restudy'
eyearray = pd.concat([studyeyearray, restudeyearray], ignore_index=True)


In [60]:
behavearray.columns

Index(['obj1', 'obj2', 'obj3', 'cuecond', 'loc1', 'x1', 'y1', 'loc2', 'x2',
       'y2', 'loc3', 'x3', 'y3', 'domRT', 'dom_resp', 'cond', 'block',
       'studytrial'],
      dtype='object')

In [61]:
def eye_behave_combo(eyearray,behavearray):
    eyebehave=eyearray.copy()
    eyecols=eyebehave.columns.tolist()
    behavecols=['x1','y1','x2','y2','x3','y3','cond', 'loc1', 'dom_resp', 'studytrial']
    allcols=eyecols+behavecols
    eyebehave=eyebehave.reindex(columns=allcols)
    order_col='studytrial'
    behavearray.sort_values(by=[order_col], inplace=True)
    behavearray.set_index(order_col, drop=False, inplace=True)
    for trial in range(0,behavearray.shape[0]):
        eyetrialevents=(eyebehave['trialnum']==trial+1)
        eyetrial=eyebehave.loc[eyetrialevents]
        for col in behavecols:
            eyetrial.loc[eyetrialevents,col]=behavearray.loc[trial+1,col]

        eyebehave.loc[eyetrialevents]=eyetrial
    return eyebehave

eyebehave = eye_behave_combo(eyearray, behavearray)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [62]:
def dist(array,x1,y1,x2,y2):
    """ distance formula for columns of coords"""
    dx=array[x1]-array[x2]
    dy=array[y1]-array[y2]
    dist=np.sqrt(dx**2+dy**2)
    return dist

def calculate_dist(eyebehave,x1,y1,name):
    """ calculate distances for start and end eye locations"""
    for x in eyebehave:
        distdict={'obj1':dist(eyebehave,x1,y1,'x1','y1'),
                        'obj2':dist(eyebehave,x1,y1,'x2','y2'),
                        'obj3':dist(eyebehave,x1,y1,'x3','y3')}

    distarray=pd.DataFrame(distdict)
    col=distarray.columns.tolist()
    distarray.columns=[c+name for c in col]
    return distarray


def loc_view(eyebehave,distarray,name):
    distarray.idxmin(axis=1)
    mindistmask=distarray.min(axis=1)<180
    distmins=distarray.loc[mindistmask]

    distminlocs=distmins.idxmin(axis=1)
    eyebehave[name]="none"
    eyebehave.loc[mindistmask,name]=distminlocs
    return eyebehave

def screenview(x,y,xmax=1920,ymax=1080):
    screen='screen'
    if x>xmax:
        screen='offscreen'
    if x<(0):
        screen='offscreen'
    if y>ymax:
        screen='offscreen'
    if y<(0):
        screen='offscreen'
    return screen

def assign_screenview(eyebehavedict,xname,yname,name):
    colname=name+'loc'
    for loc in eyebehavedict:
        screen=screenview(loc[xname],loc[yname])
        if loc[colname]=='none':
            loc[colname]=screen
        if name !='end':
            continue
        if loc['event']=='EFIX':
            loc[colname]=np.nan
    return eyebehavedict


def adjust_fix_before_blink(eyebehavedict):
    """replace fixations <100 ms before blinks"""
    tmp_dict=eyebehavedict.copy()
    new_previous_events=[]
    for i,ind in enumerate(tmp_dict):
        current_event = ind
        if i>0:
            if current_event['event']=='EBLINK':
                if previous_event['trialnum']==current_event['trialnum']:
                    if previous_event['event']=='EFIX' and previous_event['duration']<100:
                        previous_event['event']='blink'
            new_previous_events.append(previous_event)
        previous_event=ind
    new_previous_events.append(previous_event)
    return new_previous_events

def adjust_event_after_blink(new_previous_events):
    new_post_events=[]
    new_events=new_previous_events.copy()
    flag=False
    for current_event in new_events:
        event_type=current_event['event']
        current_trial=current_event['trialnum']
        if flag==True and previous_trial==current_trial:
            if event_type=='ESACC':
                event_type='blink'
            elif event_type=='EFIX':
                if current_event['duration']<100:
                    event_type='blink'
        new_post_events.append(current_event)
        flag=(event_type=='EBLINK')
        previous_trial=current_trial
    return new_post_events

def eyedict_backto_df(new_post_events):
    corrected_eyedf=pd.DataFrame(new_post_events)
    old_blink_mask=corrected_eyedf['event']!='EBLINK'
    corrected_eyedf=corrected_eyedf[old_blink_mask]
    corrected_eyedf.sort_values(['block','trialnum','start'])
    corrected_eyedf=corrected_eyedf.reset_index(drop=True)
    return corrected_eyedf

In [53]:
# calculate distances for start and end eye locations
startdistarray=calculate_dist(eyebehave,x1='xstart',y1='ystart',name='start')
enddistarray=calculate_dist(eyebehave,'xend','yend','end')

# start & end locations
eyebehave=loc_view(eyebehave,startdistarray,'startloc')
eyebehave=loc_view(eyebehave,enddistarray,'endloc')

#append start & end distances to eyebehave array
eyebehave=pd.concat([eyebehave, startdistarray, enddistarray], axis=1)

''' change df to dict'''
eyebehavedict=eyebehave.to_dict('records')
''' determine if non-loc viewing was on screen or offscreen'''
eyebehavedict=assign_screenview(eyebehavedict,'xstart','ystart','start')
eyebehavedict=assign_screenview(eyebehavedict,'xend','yend','end')

'''adjust artifacts in eye data due to blinks'''
new_previous_events=adjust_fix_before_blink(eyebehavedict)
corrected_eye_events=adjust_event_after_blink(new_previous_events)

'''put data back in df and remove old blinks'''
subcleandf=eyedict_backto_df(corrected_eye_events)

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [72]:
eyebehave.loc[2158,:]

event          ESACC
start              2
end              146
duration         146
xstart             .
ystart             .
xend          1547.7
yend           686.2
trialnum         110
block              6
sub              401
phase          study
x1              1660
y1               650
x2              1100
y2               650
x3              1380
y3               210
cond               2
loc1              18
dom_resp          18
studytrial       110
Name: 2158, dtype: object